<h1>This is the Week 3 assignment for the Capstone.</h1>

In [7]:
import pandas as pd
import numpy as np

In [ ]:
!conda install -c anaconda beautifulsoup4

<h2>Q1. We now do web scrapping on url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"</h2>

In [8]:
from bs4 import BeautifulSoup
import requests
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")
print(soup.prettify()) # print the parsed data of html

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"ff1dbaab-0815-40f9-9a0d-4f9528406a80","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":969510799,"wgRevisionId":969510799,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communicati

<h2>Convert the required table to a dataframe</h2>
<h3>1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood</h3>

In [9]:
table = soup.find('table', attrs={'class':'wikitable sortable'})
output = pd.read_html(str(table))[0]
output.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h3>2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.</h3>

In [10]:
df = output.copy()
df = df[(df['Borough']!='Not assigned')].reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h3>3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.</h3>

In [ ]:
# No code is needed because the recent wiki web page has already combined the rows on the same postal code. See the last cell for example.

<h3>4.If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.</h3>

In [11]:
# No code is needed because there is no 'Not assigned neighborhood.'
df[df.Neighbourhood == 'Not assigned']

,Postal Code,Borough,Neighbourhood


<h3>6. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.</h3>

In [12]:
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
df.shape

(103, 3)

<h1>Q2. Import csv to find out the latitude and longitude.</h1>

In [13]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h2> Merge the tables using pd.merge</h2

In [14]:
neighbourhoods = pd.merge(df,lat_long[['Postal Code', 'Latitude', 'Longitude']],on='Postal Code', how='left')
neighbourhoods.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<h1>Q3. It is time to replicate the clustering process of New York Neighbourhood to Toronto. First import libraries.</h1>

In [3]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                       

<h3>Use geopy library to get the latitude and longitude values of Toronto.</h3>

In [5]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


<h3>Create a map of Toronto with neighborhoods superimposed on top.</h3>

In [15]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Borough'], neighbourhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h3>Let us check the boroughs with text "Toronto" in it, there are 4 of them. We choose k == 4 for clustering purpose.

In [16]:
neighbourhoods.Borough.unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

Run *k*-means to cluster the neighborhood into 4 clusters.

In [17]:
# Only include data with Borough having "Toronto"
data_toronto = neighbourhoods[(neighbourhoods['Borough'].str.contains("Toronto"))].reset_index(drop=True)
data_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [18]:
# set number of clusters
kclusters = 5

toronto_clustering = data_toronto.drop(['Postal Code', 'Borough', 'Neighbourhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 4, 0, 0, 3, 0, 1, 0, 3, 4, 0, 3, 4, 0, 4, 2, 2, 2, 2,
       1, 2, 3, 1, 2, 3, 1, 2, 3, 2, 0, 0, 0, 0, 0, 0, 4], dtype=int32)

In [19]:
data_toronto["Cluster Label"] = kmeans.labels_
data_toronto["Borough_and_label"] = data_toronto['Borough'].astype(str) + '_' + data_toronto['Cluster Label'].astype(str)
data_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Label,Borough_and_label
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Downtown Toronto_0
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Downtown Toronto_0
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Downtown Toronto_0
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Downtown Toronto_0
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,East Toronto_4


In [82]:
data_toronto.Borough_and_label.unique()

array(['Downtown Toronto_0', 'East Toronto_4', 'Downtown Toronto_3',
       'West Toronto_1', 'West Toronto_3', 'Central Toronto_2',
       'Central Toronto_3'], dtype=object)

<h3> We can see that the clustering result does not fully replicate the correct borough.</h3>
Let us plot the locations on the map.

In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(data_toronto['Latitude'],data_toronto['Longitude'], data_toronto['Neighbourhood'], data_toronto['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters